In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from collections import Counter
from datetime import datetime, timedelta
from sklearn import metrics
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn import ensemble
import sklearn.metrics
import time
from datetime import datetime

%matplotlib inline                                
#no need to do plt.show()

In [2]:
train = pd.read_csv('./train.csv',parse_dates=['date'])
test = pd.read_csv('./test.csv',parse_dates=['date'],index_col = 0)
print( ' train shape:{}, columns:{} '.format(train.shape,train.columns))
print( ' test shape:{}, columns:{}'.format(test.shape,test.columns))

print(train['date'].head())
print(train.columns)
train.head()

 train shape:(14006, 3), columns:Index(['id', 'date', 'speed'], dtype='object') 
 test shape:(3504, 1), columns:Index(['date'], dtype='object')
0   2017-01-01 00:00:00
1   2017-01-01 01:00:00
2   2017-01-01 02:00:00
3   2017-01-01 03:00:00
4   2017-01-01 04:00:00
Name: date, dtype: datetime64[ns]
Index(['id', 'date', 'speed'], dtype='object')


,id,date,speed
0,0,2017-01-01 00:00:00,43.002930
1,1,2017-01-01 01:00:00,46.118696
2,2,2017-01-01 02:00:00,44.294158
3,3,2017-01-01 03:00:00,41.067468
4,4,2017-01-01 04:00:00,46.448653


In [3]:
###处理特征


#date转  年 月 日 时 周几 年内第几周 季节 ：date speed year month day hr weekday weekofyear season dayofyear
for df in (train,test): 
    df['year']  = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day']   = df['date'].dt.day
    df['hr']    = df['date'].dt.hour
    df["weekdayth"] = df['date'].dt.weekday
    df["weekofyear"] = df['date'].dt.isocalendar().week.astype("int")
    df["season"] = df['date'].dt.quarter.fillna(0).astype("int")
    df["dayofyear"] = df['date'].dt.dayofyear
    
    
for df in (train,test):
    df['y/n-weekday'] = np.nan 
    df.loc[(df.weekdayth>=5),'y/n-weekday']=1  #周六周天为1
    df.loc[(df.weekdayth<=4),'y/n-weekday']=0  #工作日为0
# #时 转 白天黑夜早晚高峰      ：date speed year month day hr weekday weekofyear season dayofyear hr-categori
# for df in (train,test):
#     df['hr-categori'] = np.nan #创建一个空字段，选择一个内容进行占位，不影响后面数据的插入
#     df.loc[(df.weekday <= 4)&(df.weekday >= 0)&(df.hr <= 9)&(df.hr >= 7), 'hr-categori'] = "3"   #工作日的早高峰7-9点
#     df.loc[(df.weekday <= 4)&(df.weekday >= 0)&(df.hr <= 19)&(df.hr >= 17), 'hr-categori'] = "3"  #工作日的晚高峰17-19点
    
#     df.loc[(df.weekday <= 4)&(df.weekday >= 0)&(df.hr >= 0)&(df.hr < 7), 'hr-categori'] = "0"
#     df.loc[(df.weekday <= 4)&(df.weekday >= 0)&(df.hr > 9)|(df.hr < 17), 'hr-categori'] = "1"
#     df.loc[(df.weekday <= 4)&(df.weekday >= 0)&(df.hr >19)&(df.hr <= 24), 'hr-categori'] = "2"

#     df.loc[(df.weekday >= 5)&(df.hr < 19)&(df.hr > 7), 'hr-categori'] = "4"                        #周末的白天
#     df.loc[(df.weekday >= 5)&(df.hr <= 7)|(df.hr >= 19), 'hr-categori'] = "5"                    #周末的晚上

    
#加香港2017 2018节假日       ：date speed year month day hr weekday weekofyear season dayofyear hr-categori y/n-holiday
public_vacation_list = [
    '20180101', '20180216','20180217','20180219','20180330','20180331','20180402','20180405',
    '20180501','20180522','20180618','20180702','20180925','20181001','20181017','20181225','20181226',
    '20170102','20170128','20170130','20170131','20170404','20170414','20170415','20170417','20170501',
    '20170503','20170530','20170701','20171002','20171005','20171028','20171225','20171226'
] 
for df in (train,test):
    df['only-ymd'] = df['date'].apply(lambda x: x.strftime('%Y%m%d'))
    df['y/n-holiday'] = df['only-ymd'].apply(lambda x: 1 if x in public_vacation_list else 0)

    
    
df_train = train.drop(columns= ['only-ymd','date','id'], axis= 1).copy()
df_test = test.drop(columns= ['only-ymd','date'], axis= 1).copy()


In [4]:
df_train.corr()['speed']    #查看相关性

speed          1.000000
year           0.015102
month         -0.028827
day           -0.019485
hr            -0.390044
weekdayth      0.115887
weekofyear    -0.026907
season        -0.022967
dayofyear     -0.030255
y/n-weekday    0.144478
y/n-holiday    0.055386
Name: speed, dtype: float64

In [5]:
df_train.head()

,speed,year,month,day,hr,weekdayth,weekofyear,season,dayofyear,y/n-weekday,y/n-holiday
0,43.002930,2017,1,1,0,6,52,1,1,1.0,0
1,46.118696,2017,1,1,1,6,52,1,1,1.0,0
2,44.294158,2017,1,1,2,6,52,1,1,1.0,0
3,41.067468,2017,1,1,3,6,52,1,1,1.0,0
4,46.448653,2017,1,1,4,6,52,1,1,1.0,0


In [6]:
df_test.head()

,year,month,day,hr,weekdayth,weekofyear,season,dayofyear,y/n-weekday,y/n-holiday
id,,,,,,,,,,
0,2018,1,1,2,0,1,1,1,0.0,1
1,2018,1,1,5,0,1,1,1,0.0,1
2,2018,1,1,7,0,1,1,1,0.0,1
3,2018,1,1,8,0,1,1,1,0.0,1
4,2018,1,1,10,0,1,1,1,0.0,1


In [10]:
#自动调参不需要分valid





X_train = df_train.drop(['speed'], axis=1)     #提取训练数据自变量
Y_train = df_train["speed"]

Y_train = Y_train.reset_index().drop('index',axis = 1)

In [11]:
X_train.head()

,year,month,day,hr,weekdayth,weekofyear,season,dayofyear,y/n-weekday,y/n-holiday
0,2017,1,1,0,6,52,1,1,1.0,0
1,2017,1,1,1,6,52,1,1,1.0,0
2,2017,1,1,2,6,52,1,1,1.0,0
3,2017,1,1,3,6,52,1,1,1.0,0
4,2017,1,1,4,6,52,1,1,1.0,0


In [12]:
Y_train.head()

,speed
0,43.002930
1,46.118696
2,44.294158
3,41.067468
4,46.448653


In [8]:
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'booster': 'gbtree',
              'objective':['reg:linear'],
              'learning_rate': [0.01,0.03,0.02], #0.01-0.2
              'max_depth': [6,8,10, None], #树的深度越大，则对数据的拟合程度越高（过拟合程度也越高）。即该参数也是控制过拟合
              'min_child_weight': [1,0.5,1.5],
              'subsample': [0.7,0.8,0.9],
              'colsample_bytree': [0.7,0.8,0.9],
              'n_estimators': [3000],
              'gamma':[0,0.1,0.2],
              
              'reg_alpha': [0,1], 
              'reg_lambda': [1,0]}


clf = RandomizedSearchCV(xgb1,
                        parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=True,scoring = "neg_mean_squared_error")
clf.fit(X_train,Y_train)
print(clf.best_estimator_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


[17:14:46] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed:    1.4s finished


XGBoostError: [17:14:46] /Users/travis/build/dmlc/xgboost/src/gbm/gbm.cc:26: Unknown gbm type r
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000011e60ea60 dmlc::LogMessageFatal::~LogMessageFatal() + 112
  [bt] (1) 2   libxgboost.dylib                    0x000000011e69b506 xgboost::GradientBooster::Create(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, xgboost::GenericParameter const*, xgboost::LearnerModelParam const*) + 198
  [bt] (2) 3   libxgboost.dylib                    0x000000011e6c0358 xgboost::LearnerConfiguration::ConfigureGBM(xgboost::LearnerTrainParam const&, std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > >, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string